In [2]:
import pandas as pd
import numpy as np
from os import listdir

In [3]:
df1 = pd.read_csv("./csv/events_up_to_01062018.csv",low_memory=False)

In [4]:
df2 = pd.read_csv("./csv/labels_training_set.csv")

In [5]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

In [6]:
def modelo(obj):
    if(pd.isnull(obj)):
        return obj
    
    list = obj.split(" ")
    if len(list) == 1:
        return obj
    if "iPhone" in list:
        return "iPhone"
    if "Samsung" in list:
        return "Samsung"
    if "Motorola" in list:
        return "Motorola"
    if "Lenovo" in list:
        return "Lenovo"
    if "Sony" in list:
        return "Sony"
    if "LG" in list:
        return "LG"
    if "Asus" in list:
        return "Asus"
    if "iPad" in list:
        return "iPad"
    if "Quantum" in list:
        return "Quantum"
    return obj

df1['model'] = df1['model'].apply(modelo)

In [7]:
df1['model'].value_counts()

iPhone              593603
Samsung             501307
Motorola            162914
LG                   24814
Sony                 15599
Lenovo               11382
iPad                  5663
Asus                  4536
Quantum               1693
Outros TV LED 15         1
Xiaomi Redmi 2           1
Name: model, dtype: int64

In [8]:
df1['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

# Analisis para los eventos distintos de convercion

In [139]:
df_a = df1

In [148]:
df_a_model = df_a.dropna(subset=['model']).loc[:,['person','model']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model_no_convercion"})


df_a_storage = df_a.dropna(subset=['storage']).loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage_no_convercion"})
        
        
df_a_condition = df_a.dropna(subset=['condition']).loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition_no_convercion"})
        
        
df_a_color = df_a.dropna(subset=['color']).loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color_no_convercion"})

df_a_search_engine = df_a.dropna(subset=['search_engine']).loc[:,['person','search_engine']].groupby('person')\
                .apply(lambda x: x['search_engine'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"search_engine"})

In [149]:
df_datos_no_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\
                                .merge(df_a_search_engine,how='left',on='person')

In [150]:
dffinal = df_datos_no_convercion

# Date

# eventos por dia x usr ( al temporario)

In [246]:

def dds(ele):# 0 -> lunes
    if(ele.month == 1):
        return (((ele.day + i) % 7) == (i + 1))
    if(ele.month == 2):
        return (((ele.day + i + 4) % 7) == (i + 1))
    if(ele.month == 3):
        return (((ele.day + i + 4) % 7) ==(i + 1))
    if(ele.month == 4):
        return (((ele.day +  1) % 7) ==(i + 1))
    if(ele.month == 5):
        return (((ele.day + 1) % 7) == (i + 1))
i = 0
df1["lunes"] = df1["timestamp"].apply(dds)
i = 1
df1["martes"] = df1["timestamp"].apply(dds)
i = 2
df1["miercoles"] = df1["timestamp"].apply(dds)
i = 3
df1["jueves"] = df1["timestamp"].apply(dds)
i = 4
df1["viernes"] = df1["timestamp"].apply(dds)
i = 5
df1["sabado"] = df1["timestamp"].apply(dds)
i = 6
df1["domingo"] = df1["timestamp"].apply(dds)


In [247]:
dffinal = dffinal.set_index("person")

# mayor cantidad de eventos separados por dia de la semaan por usr 

In [248]:
def diaMasGrande(fila):
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        aux = fila["domingo"]
    return aux
dffinal["numero mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(diaMasGrande,axis = 1)

# dia de la semana con mas eventos por usr

In [249]:
def strDiaMasGrande(fila):
    strAux = "lunes"
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        strAux = "martes"
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        strAux = "miercoles"
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        strAux = "jueves"
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        strAux = "viernes"
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        strAux = "sabado"
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        strAux = "domingo"
        aux = fila["domingo"]
    return strAux
dffinal["nombre mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(strDiaMasGrande,axis = 1)

# cuento eventos por persona

In [262]:
aux = df1[["event","person"]]
aux["aux"] = 1
aux = aux.groupby(["person","event"]).count()
aux = aux.unstack().fillna(0).reset_index().reset_index()
aux.columns = aux.columns.droplevel()
aux["person"] = aux.iloc[:, 1].values
aux = aux.drop("",axis = 1)

/home/tito/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [263]:
dffinal = dffinal.reset_index()

In [264]:
aux = pd.merge(aux,dffinal,on="person",how="inner")

In [161]:
aux.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'person', 'index',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine'],
      dtype='object')

In [178]:
# https://www.kaggle.com/ogrellier/python-target-encoding-for-categorical-features

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
   
    assert len(trn_series) == len(target)
   # assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    return add_noise(ft_trn_series, noise_level)

In [162]:
def setPred(col):
    if(col != 0):
        return 1
    else:
        return 0

In [265]:
aux = aux.dropna(subset=["storage_no_convercion"])

In [266]:
prediccion = aux["conversion"].apply(setPred)

In [267]:
aux["search_engine"] = aux["search_engine"].fillna("not")

In [268]:
aux["viewed product2"] =  target_encode(trn_series = aux["model_no_convercion"],  \
                         target=prediccion, \
                         min_samples_leaf=50,\
                         smoothing=5,\
                         noise_level=0.01)

aux["condition_no_convercion2"] =  target_encode(trn_series = aux["condition_no_convercion"],  \
                         target=prediccion, \
                         min_samples_leaf=50,\
                         smoothing=5,\
                         noise_level=0.01)

aux["color_no_convercion2"] =  target_encode(trn_series = aux["color_no_convercion"],  \
                         target=prediccion, \
                         min_samples_leaf=50,\
                         smoothing=5,\
                         noise_level=0.01)

aux["search_engine2"] =  target_encode(trn_series = aux["search_engine"],  \
                         target=prediccion, \
                         min_samples_leaf=50,\
                         smoothing=5,\
                         noise_level=0.01)

aux["nombre mas usado dia2"] =  target_encode(trn_series = aux["nombre mas usado dia"],  \
                         target=prediccion, \
                         min_samples_leaf=50,\
                         smoothing=5,\
                         noise_level=0.01)



In [269]:
aux["storage_no_convercion"] = aux["storage_no_convercion"].str.replace("GB","").replace("512MB","0.5")

In [270]:
aux = aux.drop(["level_0","nombre mas usado dia","index","model_no_convercion","condition_no_convercion","color_no_convercion","search_engine","viewed product2"],axis = 1)

In [271]:
aux.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site,person,level_0,storage_no_convercion,numero mas usado dia,condition_no_convercion2,color_no_convercion2,search_engine2,nombre mas usado dia2
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0008ed71,0,32,6.0,0.110789,0.142831,0.068132,0.110431
1,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0,00091926,1,32,86.0,0.125877,0.113719,0.068107,0.123239
2,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,00091a7a,2,16,0.0,0.108758,0.081010,0.067652,0.097313
3,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0,000ba417,3,16,147.0,0.111960,0.142824,0.134858,0.113900
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0,000c79fe,4,128,17.0,0.110775,0.062083,0.132483,0.112307


In [272]:
aux.to_csv("features0,0.csv")